In [28]:
import numpy as np 
from sklearn.cluster import DBSCAN
import Levenshtein
import matplotlib.pyplot as plt
import os
import bisect

from utils import *


In [67]:
# translate into all possible frames (6)
# can jump into a different frame only between reads -  no need (worry that same read different orientation will cause loop, but low chances with many reads long length)
# save k-mer graph in hash table


# recursion
# start only at methionine
# store k-mers in dictionary, go down most common path
# can use bloom filter for search

#  [ v for k,v in my_dict.items() if k.startswith('Date')]


In [6]:
print(os.getcwd())
t=read_fastq('../Data/HIV/CV167_1.fastq')
dict=create_kmer_dictionary(t,8)

key=list(dict.keys())
val=list(dict.values())
print(len(val))







/Users/nogaaharony/Library/CloudStorage/OneDrive-cumc.columbia.edu/CompGen/VORF/scripts
974141


In [30]:
rate_top_kmers=0.001 #trinity just uses max
rate_to_sig=0.05 #same as trinity
kmer_count_threshold=np.mean(val[:round(len(val)*rate_top_kmers)])*rate_to_sig

index_to_keep=bisect.bisect_left(val, kmer_count_threshold)
dict_clean={k:v for k,v in zip(key[0:index_to_keep], val[0:index_to_keep])}

In [40]:

start_sites=[contig(None,k, dict_clean[k]) for k in dict_clean.keys() if k.startswith('M') and '_' not in k]
start_sites.sort(key=lambda x: x.abundance, reverse=True)

k=8
protein_dict={}
for i in range(len(start_sites)):
    queue=[]
    strand=start_sites.pop()
    while True:
        if strand.string[-1]=='_':
            break
        exts=[contig(strand, key, val) for key,val in dict_clean.items() if key.startswith(strand.string[-k+1:])]
        if len(exts)==1:
            strand.string+=exts[0].string[-1]
        elif not bool(queue):
            strand=None
            break
        else:
            queue.extend(exts)
            queue.sort(key=lambda x: x.abundance, reverse=True)
            strand.string=strand.string[:-k+1]
            strand=queue.pop()
    if strand!=None:
        protein=""
        ab=0
        while strand.parent!=None:
            protein=strand.string+protein
            ab=ab+strand.abundance
            strand=strand.parent
        protein=strand.string+protein
        ab=strand.abundance+ab
        protein_dict[protein]=ab

        print(protein+":"+str(ab/(len(protein))))

# do one then generalize for other genes
# if 2 kmers identical except for one bp take bigger
# go backwards fromstop codon, mark the last start codon you see at any point, if get to another stop codon or no options defaultto the start codon
# or go back until option with highest abundance





MTRVISGTKAVHLPRK_:0.1


In [45]:

print(len(dict_clean))

start_pos={ k: dict_clean[k] for k in dict_clean.keys() if k.startswith('M') and '_' not in k}
# TODO: add edge case for start pos in first kmer
strs=[]
for k,v in start_pos.items():
    strs.append(find_path(dict_clean, k, 10,v))

strs=flatten(strs)
strs.sort(key = lambda x: x[1],reverse=True) 
print(strs)


#merge similar strings  (different base pairs, then alternative start sites)
stn=[s[0] for s in strs]
#x1,y1=np.meshgrid(stn,stn)
#veclev=np.vectorize(Levenshtein.ratio)
#compmat=veclev(x1,y1)
#compmat>0.95

# when find path need to remove all reverse complements and alternative start sites from dictionary

def lev_metric(x, y):
    i, j = int(x[0]), int(y[0])     # extract indices
    return Levenshtein.distance(stn[i], stn[j])

X = np.arange(len(stn)).reshape(-1, 1)



34026


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
def make_paths(dict, str,k,score):
    # if last letter is stop sequence, return substring and remove from all kmers from dict
    if str[-1]=='_':
        # TODO: remove instances from dict
        return str,score/(len(str)-k+1)
    # else, find all possible paths
    else:
        next={key[-1]:val for key,val in dict.items() if key.startswith(str[-k+1:])}

        return [find_path(dict, str+n,k,score+s) for n,s in next.items()]

# create paths and save them
# if there is a break save path and score up to break
# then 

In [15]:
clustering=DBSCAN(eps=2, metric=lev_metric, min_samples=2).fit(X)
clustering.labels_

array([-1,  0, -1,  1,  0, -1,  1, -1, -1,  2,  2,  3,  3,  4,  4, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1])